# Step By Step Code

In [1]:
import os
import cv2
import numpy as np
import glob


def find_white_background(threshold=0.3):
    
    img = [cv2.imread(file) for file in glob.glob("LED TV mini Data Set/*.png")]
        
    for imgArr in img:
        background = np.array([255, 255, 255])
        percent = (imgArr == background).sum() / imgArr.size
        if percent >= threshold:
            #print(percent)
            print(True)
        else:
            print(False)

In [2]:
find_white_background()

True
True
False
False
False
False
True
False
False
True
False
False


In [9]:
# Remove background of the image
import cv2
import numpy as np

# load image
img = cv2.imread('1.png')

# convert to graky
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# threshold input image as mask
mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

# negate mask
mask = 255 - mask

# apply morphology to remove isolated extraneous noise
# use borderconstant of black since foreground touches the edges
kernel = np.ones((3,3), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# anti-alias the mask -- blur then stretch
# blur alpha channel
mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

# linear stretch so that 127.5 goes to 0, but 255 stays 255
mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

# put mask into alpha channel
result = img.copy()
result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
result[:, :, 3] = mask

# save resulting masked image
cv2.imwrite('output.png', result)

# display result, though it won't show transparency
cv2.imshow("RESULT", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Add Image Background

from PIL import Image

# Opening the primary image (used in background)
img1 = Image.open(r"bg.png")

# Opening the secondary image (overlay image)
img2 = Image.open(r"output.png")

# Pasting img2 image on top of img1
# starting at coordinates (0, 0)
img1.paste(img2, (70,70), mask = img2)

# Displaying the image
img1.show()

img1 = img1.save("Final.png")

In [ ]:
# Blur the background

import cv2
import numpy as np
import Tracker_HSV as tr
from PIL import Image
import os

cv2.namedWindow('mask',cv2.WINDOW_NORMAL)
cv2.resizeWindow('mask',(500,500))
filename= 'Final.png'
cwd = os.getcwd()
name_file=os.path.splitext(filename)[0]

path_save_temp=os.path.join(cwd,'Data')
path_save_folder=os.path.join(path_save_temp,f'{name_file}_blur_data')
if not os.path.exists(path_save_folder):
    os.makedirs(path_save_folder)


img=cv2.imread(filename)
img = cv2.GaussianBlur(img,(5,5),0)
img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

file_save_blur= os.path.join(path_save_folder,'blur.png')
im_blur = cv2.GaussianBlur(img,(81,81),0)
cv2.imwrite(file_save_blur,im_blur)

xs,ys,w,h = cv2.selectROI('mask',img)
crop_img=crop_img_true=crop_img_contour=img[ys:ys+h, xs:xs+w]

if not crop_img_true.shape[0]> 1:
    crop_img_true=img

x,y,z,a,b,c=(tr.tracker(crop_img_true))

crop_img_true=cv2.cvtColor(crop_img_true,cv2.COLOR_BGR2HSV)

file_save_mask_inrange= os.path.join(path_save_folder,'mask inRange.png')
mask_inRange=cv2.inRange(crop_img_true,(x,y,z),(a,b,c))
cv2.imwrite(file_save_mask_inrange,mask_inRange)


_, threshold = cv2.threshold(mask_inRange, 250, 255, cv2.THRESH_BINARY)
Gauss_threshold =cv2.adaptiveThreshold(threshold,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,101,10)


blank_space_black= np.zeros((crop_img_true.shape[0],crop_img_true.shape[1]),np.uint8)
blank_space_black[:]=(0)

_,contours,_ = cv2.findContours(Gauss_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)


maxi=cv2.contourArea(contours[0])
c=[]

for cnt in contours:
    if cv2.contourArea(cnt)>=maxi:
        maxi=cv2.contourArea(cnt)
##        print(cv2.contourArea(cnt))
        c= cnt

file_save_contour= os.path.join(path_save_folder,'Contour.png')
cv2.drawContours(crop_img_contour, c, -1, (0, 255, 0), 5)
cv2.imwrite(file_save_contour,crop_img_contour)


file_save_poly= os.path.join(path_save_folder,'mask fill poly.png')
mask_poly=cv2.fillConvexPoly(blank_space_black,c,(255,255,255))
cv2.imwrite(file_save_poly,mask_poly)

crop_img_true=cv2.cvtColor(crop_img_true,cv2.COLOR_HSV2BGR)

file_save_mask_bitwise= os.path.join(path_save_folder,'mask bitwise and.png')
mask_bitwise_and = cv2.bitwise_and(crop_img_true,crop_img_true,mask=mask_poly)
cv2.imwrite(file_save_mask_bitwise,mask_bitwise_and)

im2= Image.open(file_save_mask_bitwise)
im2=im2.convert('RGBA')

datas=im2.getdata()
newdata=[]

for data in datas:
    if data[0]== 0 and data[1]== 0 and data[2]== 0:
        newdata.append((255,255,255,0))
    else:
        newdata.append(data)

file_save_transparent= os.path.join(path_save_folder,'transparent.png')
im2.putdata(newdata)
im2.save(file_save_transparent)

im_blur= Image.open(file_save_blur)

file_save_final= os.path.join(path_save_folder,'final.png')
im_blur.paste(im2,(xs,ys),im2)
im_blur.save(file_save_final)

im_final= Image.open(file_save_final)
im_final.show('Final Result')
cv2.waitKey(0)
cv2.destroyAllWindows()    

# Displaing all White background images with new Background

In [5]:
import cv2
import numpy as np
import glob
from PIL import Image
import Tracker_HSV as tr


def find_white_background(threshold=0.3):
    
    img = [cv2.imread(file) for file in glob.glob("LED TV mini Data Set/*.png")]
        
    for imgArr in img:
        background = np.array([255, 255, 255])
        percent = (imgArr == background).sum() / imgArr.size
        if percent >= threshold:
            #print(percent)
            print(True)     
            
            
            
            # Remove background of the image
            
            
            
            img = imgArr
            
            # convert to graky
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # threshold input image as mask
            mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]
            
            # negate mask
            mask = 255 - mask
            
            # apply morphology to remove isolated extraneous noise
            # use borderconstant of black since foreground touches the edges
            kernel = np.ones((3,3), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

            # anti-alias the mask -- blur then stretch
            # blur alpha channel
            mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

            # linear stretch so that 127.5 goes to 0, but 255 stays 255
            mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)
            
            # put mask into alpha channel
            result = img.copy()
            result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
            result[:, :, 3] = mask
            
            # save resulting masked image
            cv2.imwrite('output.png', result)
            
            
            
            # Add Image Background
            
            
            
            # Opening the primary image (used in background)
            img1 = Image.open(r"bg.png")
            
            # Opening the secondary image (overlay image)
            img2 = Image.open(r"output.png")
            
            # Pasting img2 image on top of img1
            # starting at coordinates (0, 0)
            img1.paste(img2, (70,70), mask = img2)
            
            # Displaying the image
            img1.show()
            
            img1 = img1.save("Final.png")

In [6]:
find_white_background()

True
True
True
True
